author : Ho Man Cheng
description : for training a facial type model

In [6]:
# import dataset
import pandas as pd
from util import (DATASET_PATH )
import tensorflow
from tensorflow.keras import models, layers, optimizers, callbacks, applications
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# from huggingface

from transformers import pipeline

from huggingface_hub import hf_hub_download

In [8]:
BASE_REPO = "facebook/convnextv2-tiny-1k-224"

In [ ]:
hf_hub_download(BASE)

In [16]:
import datasets

train_data = datasets.load_dataset(f"{DATASET_PATH}/face_shape")

In [24]:
train_data

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 6625
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 180
    })
})

In [47]:
label_list =  train_data["train"].features["label"].names
id2label = {}
label2id = {}
for idx, label in enumerate(label_list):
    id2label[idx] = label 
    label2id[label] = [idx]
# id2label = {0: "Invalid", 1: "Valid"}
# label2id = {val: key for key, val in train_data["train"].features["label"]}

In [49]:
from transformers import (
    TFConvNextV2ForImageClassification,
)

base_model = TFConvNextV2ForImageClassification.from_pretrained(
    BASE_REPO, num_labels=len(label_list), id2label=id2label, label2id=label2id
)

2024-04-14 18:27:35.572866: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Input to reshape is a tensor with 768000 values, but the requested shape has 4608


TypeError: InvalidArgumentError.__init__() missing 2 required positional arguments: 'op' and 'message'

In [19]:


base_model.compile(
    optimizer="adam",
    loss='sparse_categorical_crossentropy',
    metrics=["accuracy"],
)



In [34]:
splits = train_data["train"].train_test_split(test_size=0.2)
train_ds = splits["train"]
val_ds = splits["test"]


train_set = train_ds.to_tf_dataset(
    batch_size=32,
    shuffle=True,
    columns=["image", "label"],
)
val_set = val_ds.to_tf_dataset(
    columns=["image", "label"],
    shuffle=False,
    batch_size=32,
)

In [33]:

base_model.fit(
    train_set,
    validation_data=val_set,
    # callbacks=callbacks,
    epochs=250,
    batch_size=32,
)

Epoch 1/250
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


ValueError: in user code:

    File "/home/ghost/anaconda3/envs/uts-dl/lib/python3.12/site-packages/tf_keras/src/engine/training.py", line 1398, in train_function  *
        return step_function(self, iterator)
    File "/home/ghost/anaconda3/envs/uts-dl/lib/python3.12/site-packages/tf_keras/src/engine/training.py", line 1370, in run_step  *
        outputs = model.train_step(data)
    File "/home/ghost/anaconda3/envs/uts-dl/lib/python3.12/site-packages/transformers/modeling_tf_utils.py", line 1602, in train_step  *
        raise ValueError("Could not find label column(s) in input dict and no separate labels were provided!")

    ValueError: Could not find label column(s) in input dict and no separate labels were provided!
